In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as soup

In [2]:
final_data = pd.DataFrame()

In [3]:
for i in range(1, 52) :
    
    header = {'Origin': 'https://www.1mg.com',
             'Referer': 'https://www.1mg.com/categories/fitness-supplements/protein-supplements/whey-protein-98?filter=true&pageNumber={}'.format(i),
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
             }
    
    html = requests.get('https://www.1mg.com/categories/fitness-supplements/protein-supplements/whey-protein-98?filter=true&pageNumber={}'.format(i), headers=header)
    soup_obj = soup(html.content, 'lxml')
    
    product_name = []
    quantity = []
    product_rating = []
    product_rating_count = []
    product_price = []

    for item in soup_obj.findAll('div', class_ = 'col-md-3 col-sm-4 col-xs-6 style__container___1TL2R') :

        for p_name in item.findAll('div', class_ = 'style__pro-title___2QwJy') :
            product_name.append(p_name.text.strip())

        for p_quan in item.findAll('div', class_ = 'style__pack-size___2JQG7') :
            quantity.append(p_quan.text.strip())

        curr_rating = []
        for p_rate in item.findAll('span', class_ = 'CardRatingDetail__weight-700___27w9q') :
            curr_rating.append(p_rate.text.strip())
        if len(curr_rating) > 0 :
            product_rating += curr_rating
        else :
            product_rating.append('0.0')

        curr_rating_count = []
        for p_rate_count in item.findAll('span', class_ = 'CardRatingDetail__ratings-header___2yyQW') :
            curr_rating_count.append(p_rate_count.text.strip())
        if len(curr_rating_count) > 0 :
            product_rating_count += curr_rating_count
        else :
            product_rating_count.append('0 ratings')

        for p_price in item.findAll('div', class_ = 'style__price-tag___cOxYc') :
            product_price.append(p_price.text.strip())
            
    temp_price = []
    for pp in product_price :
        temp_price.append(pp.split('₹')[1])

    temp_rating = []
    for rat in product_rating_count :
        temp_rating.append(rat.split(' ')[0])
        
    diction = {'product' : product_name, 'quantity' : quantity, 
               'rating' : product_rating, 'number of ratings' : temp_rating,
               'price' : temp_price}
    
    curr_data = pd.DataFrame(diction)
    
    final_data = final_data.append(curr_data, ignore_index = True)

In [4]:
final_data

,product,quantity,rating,number of ratings,price
0,Optimum Nutrition (ON) Gold Standard 100% Whey...,jar of 2 lb Powder,4.1,113,3099
1,MuscleBlaze MB Biozyme Performance Whey Protei...,jar of 2 kg Powder,3.7,7,4599
2,Fast&Up 100% Whey Isolate Rich Chocolate,sachet of 31 gm Serving,0.0,0,148
3,IsoPure Low Carb 100% Whey Protein Isolate Cho...,box of 3 lb Powder,3.9,8,6459
4,GNC Pro Performance 100% Whey Protein Vanilla ...,jar of 2 kg Powder,3.7,9,4499
...,...,...,...,...,...
2101,Bolt Whey Isolate Powder with Shaker Free Pied...,jar of 2 lb Powder,0.0,0,5199
2102,Bolt Whey Protein Powder with Shaker Free Mada...,jar of 2 lb Powder,0.0,0,4499
2103,Bolt Whey Isolate Powder with Shaker Free Mada...,jar of 5 lb Powder,0.0,0,10399
2104,Bolt Whey Protein Powder with Shaker Free Mada...,jar of 5 lb Powder,0.0,0,8999


In [5]:
final_data = final_data.astype({'price' : float, 'rating' : float, 'number of ratings' : int})

In [6]:
final_data.drop_duplicates(subset = ['product'], inplace=True)

In [7]:
final_data.shape

(1244, 5)

In [8]:
brands = []
for p in final_data['product'] :
    if p.startswith('Dr') :
        temp_str = p.split(' ')[0]+p.split(' ')[1]
        brands.append(temp_str)
    else :
        temp_str = p.split(' ')[0]
        brands.append(temp_str)

In [9]:
len(brands)

1244

In [10]:
final_data['keyword'] = brands

In [11]:
final_data

,product,quantity,rating,number of ratings,price,keyword
0,Optimum Nutrition (ON) Gold Standard 100% Whey...,jar of 2 lb Powder,4.1,113,3099.0,Optimum
1,MuscleBlaze MB Biozyme Performance Whey Protei...,jar of 2 kg Powder,3.7,7,4599.0,MuscleBlaze
2,Fast&Up 100% Whey Isolate Rich Chocolate,sachet of 31 gm Serving,0.0,0,148.0,Fast&Up
3,IsoPure Low Carb 100% Whey Protein Isolate Cho...,box of 3 lb Powder,3.9,8,6459.0,IsoPure
4,GNC Pro Performance 100% Whey Protein Vanilla ...,jar of 2 kg Powder,3.7,9,4499.0,GNC
...,...,...,...,...,...,...
2095,Proathlix Xloaded 80.6% Whey Protein Powder Ca...,jar of 1 kg Powder,0.0,0,3750.0,Proathlix
2098,Bolt Whey Protein Powder with Shaker Free Pied...,jar of 5 lb Powder,0.0,0,8999.0,Bolt
2099,Bolt Whey Isolate Powder with Shaker Free Pied...,jar of 5 lb Powder,0.0,0,10399.0,Bolt
2102,Bolt Whey Protein Powder with Shaker Free Mada...,jar of 2 lb Powder,0.0,0,4499.0,Bolt


In [13]:
# final_data.to_csv('protein_supplements_data_final.csv', index=False)